In [580]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
pd.set_option("display.max_columns", 100)
import os
import geojson
from geojson import Feature, FeatureCollection
import requests
from bs4 import BeautifulSoup
import json
import requests
from datetime import datetime
from geopy import geocoders

In [115]:
url = "https://api.covid19india.org/csv/latest/raw_data.csv"
r = requests.get(url)
f = open('../data/26042020/raw_data.csv','w')
f.write(r.text)
f.close()

### Clusters

In [641]:
df = pd.read_csv("../data/26042020/raw_data.csv")
df.dropna(subset=["Date Announced"], inplace=True)
df.rename(columns={"Contracted from which Patient (Suspected)":"contracted_from", "Current Status":"status", "Notes":"notes",
                   "Detected City":"city", "Detected District":"district", "Detected State": "state","State code":"state_code",
                   "Date Announced":"date", "Type of transmission":"transmission_type", "Age Bracket":"age", "Gender":"gender",
                   "State Patient Number": "state_pid", "Patient Number":"pid"}, inplace=True)
df = df[["pid", "date", "city", "district", "state", "contracted_from", "transmission_type", "notes"]]
total_case_count = len(df)


In [642]:
## Get root node of "contracted_from"
for i in range(len(df)):
    dft = df.iloc[i]
    cf = dft["contracted_from"] #.values[0]

    while cf==cf: 
        if cf.startswith("https"):
            break
        if ((cf!="E0") and (cf!="E1")):
            pid = cf.split("P")[1]
            if ", " in pid:
                pid = pid.split(", ")[0]
            elif "," in pid:
                pid = pid.split(",")[0]
                
            cfpid = int(pid)
            dft = df[df["pid"]==cfpid].copy()
            df.at[i, "derived_contracted_from"] = cf
            cf = dft["contracted_from"].values[0]
        else:
            df.at[i, "derived_contracted_from"] = cf
            cf = np.nan

In [647]:
for i in range(len(df)):
    cf = df.iloc[i]["contracted_from"]
    dcf = df.iloc[i]["derived_contracted_from"]
    notes = df.iloc[i]["notes"]
    district = df.iloc[i]["district"]
    state = df.iloc[i]["state"]

    # IF both notes and contracted from are empty, skip this entry
    if ((cf!=cf) and (notes!=notes)):
        continue

    if ((cf == "E0") or (cf == "P4862") or (cf == "P531") ):
        df.at[i, "cluster"] = "Delhi Religious meeting"
        df.at[i, "cluster_district"] = "New Delhi"
        df.at[i, "cluster_state"] = "Delhi"

    elif ((cf == "P689") or (cf == "P1215") ):
        df.at[i, "cluster"] = "Mysuru Pharmaceutical industry"
        df.at[i, "cluster_district"] = "Mysore"
        df.at[i, "cluster_state"] = "Karnataka"
    
    elif cf == "P182":
        df.at[i, "cluster"] = "Punjab Preacher"
        df.at[i, "cluster_district"] = "Shaheed Bhagat Singh Nagar"
        df.at[i, "cluster_state"] = "Punjab"
    
    elif cf == "P20410":
        df.at[i, "cluster"] = "Bengaluru scrap segregation worker"
        df.at[i, "cluster_district"] = "Bangalore"
        df.at[i, "cluster_state"] = "Karnataka"
    
    elif cf == "P6":
        df.at[i, "cluster"] = "Italian tourists in Rajasthan"
        df.at[i, "cluster_district"] = "Jaipur"
        df.at[i, "cluster_state"] = "Rajasthan"
    
    elif cf == "P301":
        df.at[i, "cluster"] = "Thai national in Tamil Nadu"
        df.at[i, "cluster_district"] = "Erode"
        df.at[i, "cluster_state"] = "Tamil Nadu"
    
    elif cf == "E1":
        df.at[i, "cluster"] = "Contact with UK returnee"
        df.at[i, "cluster_district"] = "Gautam Buddha Nagar"
        df.at[i, "cluster_state"] = "Uttar Pradesh"
    
    elif cf == "P2868":
        df.at[i, "cluster"] = "Dubai returnee, hosted feast for 1500 people"
        df.at[i, "cluster_district"] = "Morena"
        df.at[i, "cluster_state"] = "Madhya Pradesh"
    
    elif cf == "P10454":
        df.at[i, "cluster"] = "Doctor from Bethany Hospital"
        df.at[i, "cluster_district"] = "East Khasi Hills"
        df.at[i, "cluster_state"] = "Meghalaya"
    
    else:
        if is_in_travel_list(notes):
            travelled_from_place = is_in_travel_list(notes)
            df.at[i, "cluster"] = "Travel History"
            df.at[i, "cluster_district"] = travelled_from_place
            df.at[i, "cluster_state"] = travelled_from_place

        elif is_in_family_list(notes):
            df.at[i, "cluster"] = "Family member"
        
        elif is_in_contact_list(notes):
            df.at[i, "cluster"] = "Contact"
        
        elif is_in_healthcare_list(notes):
            df.at[i, "cluster"] = "Healthcare worker"
        
        elif is_in_worker_list(notes):
            df.at[i, "cluster"] = "Domestic Worker"
        
        elif is_in_no_travel_history_list(notes):
            df.at[i, "cluster"] = "No travel history"
        
        elif is_in_iran_evacuee_list(notes):
            df.at[i, "cluster"] = "Iran evacuee"
            df.at[i, "cluster_district"] = "Iran"
            df.at[i, "cluster_state"] = "Iran"

        elif is_in_delhi_religious_list(notes):
            df.at[i, "cluster"] = "Delhi Religious meeting"
            df.at[i, "cluster_district"] = "New Delhi"
            df.at[i, "cluster_state"] = "Delhi"
            df.at[i, "contracted_from"] = "E0"
            df.at[i, "derived_contracted_from"] = "E0"
        
        elif is_in_pharma_company_list(notes):
            df.at[i, "cluster"] = "Mysuru Pharmaceutical industry"
            df.at[i, "cluster_district"] = "Mysore"
            df.at[i, "cluster_state"] = "Karnataka"
            df.at[i, "contracted_from"] = "E1"
            df.at[i, "derived_contracted_from"] = "E1"

        elif is_in_misc_list(notes):
            cluster_category = is_in_misc_list(notes)
            df.at[i, "cluster"] = cluster_category

        elif is_in_skip_list(notes):
            continue;
            
        else:
            print(notes)
            continue;

In [666]:
## One time
"""
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
geolocator = Nominatim(user_agent="myGeocoder")

cluster_district_mapping = {"Saudi":"Saudi Arabia", "Evacuees*":"Iran", "Abhudhabi":"Abu dhabi", "WB":"West Bengal"}
cluster_district_skip = ["to", "the", "Asansole"]
dfcll = pd.DataFrame()
idx = 0
for loc in df["cluster_district"].unique()[101:]:
    if not pd.isnull(loc) and loc not in cluster_district_skip:
        if loc in cluster_district_mapping.keys():
            loc = cluster_district_mapping[loc]

        try:
            location = geolocator.geocode(loc, timeout=10) #, geometry="geojson")
            print(loc + " " + str(location.latitude), str(location.longitude))
            dfcll.at[idx, "location"] = loc
            dfcll.at[idx, "latitude"] = location.latitude
            dfcll.at[idx, "longitude"] = location.longitude
        except GeocoderTimedOut as e:
            dfcll.at[idx, "location"] = loc
            dfcll.at[idx, "latitude"] = np.nan
            dfcll.at[idx, "longitude"] = np.nan
            print(loc + " Error: geocode failed on input %s with message %s"%(my_address, e.message))

        idx+=1
dfcll.to_csv("../data/extracted_data_api/code_data/cluster_latitude_longitude.csv", index=False)            
"""
print("One time")

West Bengal 22.9964948 87.6855882
Dehradun 30.3255646 78.0436813
Pollachi 10.669422350000001 77.01017959253107
Surat 45.9383 3.2553
Hindupur 13.826383 77.4937723
Maharashtra 19.531932 76.0554568


In [673]:
geolocator = Nominatim(user_agent="district_locator")
district_in_raw_data_list = df["district"].unique()
district_skip = ["Italians*", "Other States*", "Evacuees*", "Other Region*", ]
district_mapping = {}

dfll = pd.DataFrame()
idx = 0

for district in district_in_raw_data_list:
    if not pd.isnull(district) and district not in district_skip:
        if district in district_mapping.keys():
            district = district_mapping[district]
        
        print(district)

Thrissur
Alappuzha
Kasaragod
East Delhi
Hyderabad
Agra
South West Delhi
Ghaziabad
West Delhi
Leh
Kancheepuram
Pathanamthitta
North Delhi
Jammu
Bengaluru
Kannur
Pune
Amritsar
Ernakulam
Kottayam
Jaipur
Mumbai
Nagpur
S.P.S. Nellore
Lucknow
Kalaburagi
New Delhi
Thiruvananthapuram
Ahmadnagar
Thane
Raigarh
Yavatmal
Idukki
Aurangabad
Dehradun
Malappuram
Khordha
South Delhi
Gurugram
Kargil
Mahe
Gautam Buddha Nagar
Kolkata
Chennai
Ratnagiri
Srinagar
Jhunjhunu
North East Delhi
Chandigarh
Kodagu
Prakasam
Raipur
Kheri
Shahid Bhagat Singh Nagar
Rajkot
Surat
Visakhapatnam
North West Delhi
S.A.S. Nagar
Ahmadabad
Vadodara
Bhilwara
Faridabad
Panipat
Kangra
Jabalpur
Karimnagar
North 24 Parganas
Panchkula
Gandhinagar
Moradabad
Erode
Hoshiarpur
Mysuru
Sonipat
Kachchh
East Godavari
Krishna
Varanasi
Dharwad
Coimbatore
Munger
Patna
Bhopal
Bhadradri Kothagudem
Uttara Kannada
Kozhikode
Tirunelveli
Jodhpur
Pali
Sikar
Palghar
Palwal
Kanpur Nagar
Pilibhit
Pratapgarh
Tiruppur
Madurai
Jaunpur
Sangli
Satara
Imphal W

In [681]:
loc = "East Delhi"
try:
    location = geolocator.geocode(loc, timeout=10, geometry="geojson")
    print(loc + " " + str(location.latitude), str(location.longitude))
    dfll.at[idx, "district"] = loc
    dfll.at[idx, "latitude"] = location.latitude
    dfll.at[idx, "longitude"] = location.longitude
except GeocoderTimedOut as e:
    dfll.at[idx, "district"] = loc
    dfll.at[idx, "latitude"] = np.nan
    dfll.at[idx, "longitude"] = np.nan
    print(loc + " Error: geocode failed on input %s with message %s"%(my_address, e.message))

East Delhi 28.6204767 77.30918071344418


In [682]:
location.raw

{'place_id': 235709377,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 1942588,
 'boundingbox': ['28.584857', '28.6558204', '77.2685234', '77.3420594'],
 'lat': '28.6204767',
 'lon': '77.30918071344418',
 'display_name': 'East Delhi, Delhi, India',
 'class': 'boundary',
 'type': 'administrative',
 'importance': 0.628002769416701,
 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png',
 'geojson': {'type': 'Polygon',
  'coordinates': [[[77.2685234, 28.6387995],
    [77.2699409, 28.6317816],
    [77.2713303, 28.6284904],
    [77.277643, 28.6192496],
    [77.2782298, 28.6184063],
    [77.2848849, 28.6088421],
    [77.2920584, 28.5946395],
    [77.2982764, 28.584857],
    [77.3001722, 28.5868739],
    [77.3015404, 28.5868763],
    [77.302858, 28.586664],
    [77.3044219, 28.5857974],
    [77.3088577, 28.5893282],
    [77.3101587, 28.5907418],
    [77.3111384, 28.5919458],

In [646]:
def is_in_pharma_company_list(notes):
    pharma_company_list = ["Pharmaceutical company employee", "Pharma Company Worker"]
    for f in pharma_company_list:
        if f in notes:
            return True
    return False

def is_in_delhi_religious_list(notes):
    is_in_delhi_religious_list = ["Attended Delhi Religious Conference"]
    for f in is_in_delhi_religious_list:
        if f in notes:
            return True
    return False
    
def is_in_travel_list(notes):
    travel_list = ["Travelled from ", "Travelled to ", "Tavelled from ", "Travel history to ", "Travel History to ",
                  "foreign travel from ", "travelled to ", "Returned from ", "Travel HIstory to ",
                  "History of travel to ", "Evacuees from "]
    
    travel_place_unknown_list = ["No history of travel", "Travel history", "travel history", "Foreign Travel",
                                ]
    
    for f in travel_list:
        if f in notes:
            travelled_from_place = notes.split(f)[1].split(" ")[0]
            if travelled_from_place in ["United", "New", "Sri", "Abu", "West", "Uttar"]:
                travelled_from_place = notes.split(f)[1].split(" ")[0] + " " + notes.split(f)[1].split(" ")[1] 
            if "," in travelled_from_place:
                travelled_from_place = travelled_from_place.split(",")[0]
            if "." in travelled_from_place:
                travelled_from_place = travelled_from_place.split(".")[0]
            if ";" in travelled_from_place:
                travelled_from_place = travelled_from_place.split(";")[0]
            if ")" in travelled_from_place:
                travelled_from_place = travelled_from_place.split(")")[0]
            return travelled_from_place
    
    for f in travel_place_unknown_list:
        if f in notes:
            return np.nan

    return False

def is_in_iran_evacuee_list(notes):
    iran_evacuee_list = ["Evacuees from Iran", "Evacuee from Iran"]
    for f in iran_evacuee_list:
        if f in notes:
            return True
    return False

def is_in_misc_list(notes):
    misc_list = ["Co-passenger", "Cab driver",  "Indian Army jawan", "Travelled in a car with",
                "Supertech Capetown, Sector 74, NOIDA", "Tourist", "BSF officer" , "Dharavi", "Domestic Travel",
                "SARI", "Police", "Journalist", "Influenza like illness", "Conatct with patient", "Influenza like Illness"]
    for f in misc_list:
        if f in notes:
            return f
    return False

def is_in_no_travel_history_list(notes):
    no_travel_history_list = ["No travel history", "No Travel History", "No foreign travel",
                             "No history of travelling abroad",]
    for f in no_travel_history_list:
        if f in notes:
            return True
    return False

def is_in_healthcare_list(notes):
    healthcare_list = ["Doctor", "Staff of Private Hospital", "Nurse", "Hospital staff", "Health worker",
                      "Sanitation worker", "Attendent", "Hospital Staff"]
    for f in healthcare_list:
        if f in notes:
            return f
    return False
    
def is_in_skip_list(notes):
    skip_list = ["Factory Manager contact with businessman from Italy", "Was in contact with",
                "Have international history and were brought in from the Mumbai Airport", "Sector 78; Contracted from his boss",
                 "Reached Chennai MGR Central from Delhi via Train on 12.03.2020", "Travelled to UK", "Details awaited",
                 "Bangar Hospital Compounder","Admitted on Swine Flu suspicion; COVID19 confirmed",
                 "Worked at a beauty salon that Chandigarh’s first coronavirus patient visited",
                 "Landed in Mangalore airport, had been to Kasturba Medical College Attavar",
                 "No international travel history, has travelled to  Bilaspur by train",
                 "One more person, who is under observation at Dehradun's Doon Hospita", "Details Awaited",
                 "Sonipat", "Anganwadi Worker, Visited Velha, Attended Marriage, Tested Positive on 24th March, 2020",
                 "Native of", "details awaited", "Travel history (first case from Ramganj?)", "Travelled with ",
                 "Matched ", "A 50-year-old man, a railway guard, tested positive at the All India Institute of Medical Sciences, Bhopal",
                 "Match", "IPD patient at of Bangar Hospital", "earlier tested positive", "Woman in 40s",
                 "Worked at", "Patient is from", "Admit Noble Hospital on 2nd April, 2020", "Further Details not known",
                 "From ", "Belongs to", "Admitted to", "Admit to", "Visited", "Organized Medical camp with participation of NRIs",
                 "The detailed investigation is under process", "admit", "confimed positive", "No Info", "Classified as",
                 "They work at a bakery in Jawaharpur, Dera Bassi", "Source of", "Travelle from", "Returned to",
                 "ambulance", "Travelld from", "As per MohFW update", "Conference", "History of travel from Hidupur on 2 wheeler on 7/4/2020",
                 "Neighour of", "WB returnees", "Kolkata Returnees", "New case", "Trancing under process"
                ]
    for f in skip_list:
        if f in notes:
            return True
    return False
    
def is_in_family_list(notes):
    family_list = ["Father of", "Mother of", "Wife of", "Son of", "Daughter of", "Brother of", "Sister of", "Son in law of",
                  "Daughter in law of", "Close contact of", "Family members of" ,"Relative of", 
                  "Friends of", "Friend of", "First contacts", "family members", "Family members", "Spouse of",
                  "Father", "Mother", "Related to", "Employee of", "Contact of", "contact of", "husband",
                  "Daughter in Law", "Uncle of", "Aunt of", "Close relative of", "Relatives of", "Grandmother of",
                  "Family member of", "Cousin of", "Brother-in-law of", "Husband of", "Sister in law of", "Family",
                   "family", "relative", "Sister-in-law of"
                  ]
    for f in family_list:
        if f in notes:
            return True
    return False

def is_in_contact_list(notes):
    contact_list = ["Close contact of", "neighbour of", "Friends of", "Friend of", "First contacts",
                    "Employee of", "Contact of", "contact of", "Contact", "contact", "Local transmission",
                   "locally transmitted", "Local Transmission", "Tenant of", "Neighbour of", "neighbour of",
                   "Neighbor of", "Co-worker", "Came in touch with a positive person"]
    for f in contact_list:
        if f in notes:
            return True
    return False

def is_in_worker_list(notes):
    worker_list = ["Cook of", "Domestic worker of"]
    for f in worker_list:
        if f in notes:
            return True
    return False

### % of districts without the disease

In [48]:
dfcc = pd.read_csv("../data/extracted_data_api/code_data/districtwise_case_death_growth_density_withDelhiSeparate.csv")
total_population = dfcc["population"].sum()
per_outbreak_free_districts = np.round(len(dfcc[dfcc["cases"]==0])/len(dfcc)*100)
per_outbreak_free_population = np.round(dfcc[dfcc["cases"]==0]["population"].sum()/total_population*100)
df = pd.DataFrame()
for i in range(len(dfcc)):
    district_name = dfcc.iloc[i]["district"].replace(" ","-") + ", " + dfcc.iloc[i]["state_code"]
    df.at[i, "id"] = "district."+district_name
    df.at[i, "value"] = int(dfcc.iloc[i]["cases"])+1
    df.at[i, "district"] = district_name
    df.at[i, "per_outbreak_free_districts"] = per_outbreak_free_districts
    df.at[i, "per_outbreak_free_population"] = per_outbreak_free_population
df["value"] = df["value"].astype(int)
df["per_outbreak_free_districts"] = df["per_outbreak_free_districts"].astype(int)
df["per_outbreak_free_population"] = df["per_outbreak_free_population"].astype(int)
# Shuffling
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv("../data/extracted_data_api/code_data/outbreak_free_districts.csv", index=False)
df

,id,value,district,per_outbreak_free_districts,per_outbreak_free_population
0,"district.Sahibganj, JH",1,"Sahibganj, JH",44,31
1,"district.Kamrup, AS",2,"Kamrup, AS",44,31
2,"district.Buldana, MH",12,"Buldana, MH",44,31
3,"district.Kasaragod, KL",169,"Kasaragod, KL",44,31
4,"district.Karnal, HR",7,"Karnal, HR",44,31
5,"district.Haora, WB",24,"Haora, WB",44,31
6,"district.Dindori, MP",1,"Dindori, MP",44,31
7,"district.Kiphire, NL",1,"Kiphire, NL",44,31
8,"district.Baksa, AS",1,"Baksa, AS",44,31
9,"district.Alwar, RJ",8,"Alwar, RJ",44,31


### Recovery rate

In [112]:
dfc = pd.read_csv("../data/extracted_data_api/statewise_daily_confirmed.csv")
del dfc["TT"], dfc["Unnamed: 39"]
state_code_list = dfc.columns[1:]
dfcc = dfc.set_index("date").cumsum().reset_index()

dfr = pd.read_csv("../data/extracted_data_api/statewise_daily_recovered.csv")
del dfr["TT"], dfr["Unnamed: 39"]
l = [["date"], [col+"_r" for col in dfr.columns if col in state_code_list]]
dfr.columns = [item for sublist in l for item in sublist]
dfr = dfr.set_index("date").cumsum().reset_index()

dfcr = dfcc.set_index("date").join(dfr.set_index("date")).reset_index()
dfrr = dfcc.copy()
for col in state_code_list:
    dfrr[col] = np.round(dfcr[col+"_r"] / dfcr[col],2)
dfrr.dropna(axis=1, how="all", inplace=True)
### Replace nan with -1, handle in color_Scale in javascript
dfrr.fillna(value=-1, inplace=True)
### Concat total case count at top of table
dfrr = pd.concat([pd.DataFrame(dfc.sum(axis=0, numeric_only=True)).transpose(), dfrr], ignore_index=True, axis=0, sort=True)
l = [["date"], [col for col in dfrr.columns[:len(dfrr.columns)-1]]]
columns = [item for sublist in l for item in sublist]
dfrr = dfrr[columns]
dfrr.at[pd.isnull(dfrr["date"]), "date"] = "total_count"
dfrr.to_csv("../data/extracted_data_api/code_data/state_recovery_rate.csv", index=False)
dfrr

,date,AN,AP,AR,AS,BR,CH,CT,DD,DL,DN,GA,GJ,HP,HR,JH,JK,KA,KL,LA,LD,MH,ML,MN,MP,MZ,NL,OR,PB,PY,RJ,SK,TG,TN,TR,UP,UT,WB
0,total_count,17.00,757.00,1.0,35.00,126.00,27.00,36.00,0.0,2156.00,0.0,7.00,2178.00,39.00,255.00,46.00,380.00,418.00,426.00,18.00,0.0,5218.00,12.0,2.0,1552.00,1.0,0.0,79.00,251.00,7.00,1735.00,0.0,928.00,1596.00,2.0,1337.00,46.00,392.00
1,14-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,-1.00,-1.00,NaN,0.14,NaN,-1.00,-1.00,-1.00,0.00,-1.00,0.00,0.00,0.16,-1.00,NaN,0.00,-1.0,-1.0,-1.00,-1.0,NaN,-1.00,0.00,-1.00,0.33,NaN,0.00,0.00,-1.0,0.33,-1.00,-1.00
2,15-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,-1.00,-1.00,NaN,0.29,NaN,-1.00,-1.00,-1.00,0.00,-1.00,0.00,0.00,0.12,-1.00,NaN,0.00,-1.0,-1.0,-1.00,-1.0,NaN,-1.00,0.00,-1.00,0.75,NaN,0.33,0.00,-1.0,0.31,-1.00,-1.00
3,16-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,-1.00,-1.00,NaN,0.29,NaN,-1.00,-1.00,-1.00,0.00,-1.00,0.00,0.00,0.11,-1.00,NaN,0.00,-1.0,-1.0,-1.00,-1.0,NaN,0.00,0.00,0.00,0.75,NaN,0.25,1.00,-1.0,0.31,0.00,-1.00
4,17-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,-1.00,-1.00,NaN,0.25,NaN,-1.00,-1.00,-1.00,0.00,-1.00,0.00,0.00,0.11,-1.00,NaN,0.00,-1.0,-1.0,-1.00,-1.0,NaN,0.00,0.00,0.00,0.75,NaN,0.20,1.00,-1.0,0.33,0.00,0.00
5,18-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,-1.00,-1.00,NaN,0.20,NaN,-1.00,-1.00,-1.00,0.00,-1.00,0.00,0.00,0.11,0.00,NaN,0.00,-1.0,-1.0,-1.00,-1.0,NaN,0.00,0.00,0.00,0.43,NaN,0.08,0.50,-1.0,0.29,0.00,0.00
6,19-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,0.00,0.00,NaN,0.21,NaN,-1.00,0.00,-1.00,0.00,-1.00,0.00,0.00,0.11,0.00,NaN,0.00,-1.0,-1.0,-1.00,-1.0,NaN,0.00,0.00,0.00,0.33,NaN,0.06,0.33,-1.0,0.45,0.00,0.00
7,20-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,0.00,0.00,NaN,0.25,NaN,-1.00,0.00,0.00,0.00,-1.00,0.00,0.07,0.08,0.00,NaN,0.00,-1.0,-1.0,0.00,-1.0,NaN,0.00,0.00,0.00,0.18,NaN,0.05,0.33,-1.0,0.38,0.00,0.00
8,21-Mar-20,-1.00,0.00,-1.0,-1.00,-1.00,0.00,0.00,NaN,0.19,NaN,-1.00,0.00,0.00,0.00,-1.00,0.00,0.05,0.06,0.00,NaN,0.00,-1.0,-1.0,0.00,-1.0,NaN,0.00,0.00,0.00,0.12,NaN,0.05,0.17,-1.0,0.32,0.00,0.00
9,22-Mar-20,-1.00,0.00,-1.0,-1.00,0.00,0.00,0.00,NaN,0.19,NaN,-1.00,0.00,0.00,0.00,-1.00,0.00,0.04,0.04,0.00,NaN,0.00,-1.0,-1.0,0.00,-1.0,NaN,0.00,0.00,0.00,0.11,NaN,0.04,0.11,-1.0,0.30,0.00,0.00


### topojson to geojson

In [711]:
os.listdir("../data/extracted_data_api/code_data/state_geojson_data_howIndiaLives/")

['odisha.json',
 'telangana.json',
 'meghalaya.json',
 'karnataka.json',
 'haryana.json',
 'bihar.json',
 'andhrapradesh.json',
 'jammukashmir.json',
 'westbengal.json',
 'kerala.json',
 'chhattisgarh.json',
 'andamannicobarislands.json',
 'jharkhand.json',
 'ladakh.json',
 'uttarpradesh.json',
 'mizoram.json',
 'lakshadweep.json',
 'nagaland.json',
 'tamilnadu.json',
 'dadranagarhaveli.json',
 'delhi.json',
 'puducherry.json',
 'madhyapradesh.json',
 'arunachalpradesh.json',
 'uttarakhand.json',
 'manipur.json',
 'tripura.json',
 'gujarat.json',
 'goa.json',
 'assam.json',
 'maharashtra.json',
 'punjab.json',
 'sikkim.json',
 'rajasthan.json',
 'chandigarh.json',
 'himachalpradesh.json']

In [727]:
#https://gist.github.com/perrygeo/1e767e42e8bc54ad7262
#https://github.com/sgillies/topojson/blob/master/topojson.py

import json
import sys
from topojson_geometry import geometry
from shapely.geometry import asShape
from shapely.geometry import shape

dfll = pd.DataFrame()
idx = 0

for f in os.listdir("../data/extracted_data_api/code_data/state_geojson_data_howIndiaLives/"):
    state_name = f.split(".json")[0]
    topojson_path = "../data/extracted_data_api/code_data/state_geojson_data_howIndiaLives/"+f
    geojson_path = "../data/extracted_data_api/code_data/geojson_map/"+f

    with open(topojson_path, 'r') as fh:
        f = fh.read()
        topology = json.loads(f)

    features = topology['objects'][state_name+"_district"]['geometries']
    scale = topology['transform']['scale']
    trans = topology['transform']['translate']

    with open(geojson_path, 'w') as dest:
        fc = {'type': "FeatureCollection", 'features': []}

        for id, tf in enumerate(features):
            f = {'id': id, 'type': "Feature"}
            f['properties'] = tf['properties'].copy()

            geommap = geometry(tf, topology['arcs'], scale, trans)
            geom = asShape(geommap).buffer(0)
            assert geom.is_valid
            f['geometry'] = geom.__geo_interface__

            fc['features'].append(f) 

        # Save geojson file
        dest.write(json.dumps(fc))
    
        # Compute district center
        for d in range(len(fc["features"])):
            geom = shape(fc["features"][d]["geometry"])    
            district_bounds = geom.bounds
            district_name = fc["features"][d]["properties"]["district"]
            if district_name in district_name_mapping:
                district_name_mapped = district_name_mapping[district_name]
                district_name = district_name_mapped
                
            if district_name in district_name_resolution:
                district_name_resolved = resolve_district_name(state_name, district_name)
                district_name = district_name_resolved
            
            dfll.at[idx, "district"] = district_name
            dfll.at[idx, "latitude"] = geom.centroid.y
            dfll.at[idx, "longitude"] = geom.centroid.x

            idx+=1
dfll.to_csv("../data/extracted_data_api/code_data/district_center_latitude_longitude.csv", index=False)

In [ ]:
##https://gis.stackexchange.com/questions/207731/generating-random-coordinates-in-multipolygon-in-python
import random
from shapely.geometry import Point

def generate_random(number, polygon):
    list_of_points = []
    minx, miny, maxx, maxy = polygon.bounds
    counter = 0
    while counter < number:
        pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(pnt):
            list_of_points.append(pnt)
            counter += 1
    return list_of_points

In [719]:
state_name_mapping = {'odisha':'Odisha', 'telangana':'Telangana', 'meghalaya':'Meghalaya','karnataka':'Karnataka',
                     'haryana':'Haryana','bihar':'Bihar','andhrapradesh':'Andhra Pradesh','jammukashmir':'Jammu and Kashmir',
                     'westbengal':'West Bengal','kerala':'Kerala','chhattisgarh':'Chhattisgarh','andamannicobarislands':'Andaman and Nicobar Islands',
                     'jharkhand':'Jharkhand','ladakh':'Ladakh','uttarpradesh':'Uttar Pradesh','mizoram':'Mizoram','lakshadweep':'Lakshadweep',
                     'nagaland':'Nagaland','tamilnadu':'Tamil Nadu','dadranagarhaveli':'Dadra and Nagar Haveli','delhi':'Delhi',
                     'puducherry':'Puducherry','madhyapradesh':'Madhya Pradesh','arunachalpradesh':'Arunachal Pradesh','uttarakhand':'Uttarakhand',
                     'manipur':'Manipur','tripura':'Tripura','gujarat':'Gujarat','goa':'Goa','assam':'Assam','maharashtra':'Maharashtra',
                     'punjab':'Punjab','sikkim':'Sikkim','rajasthan':'Rajasthan','chandigarh':'Chandigarh','himachalpradesh':'Himachal Pradesh'}

state_name_mapping_howindialives = {"Jammu & Kashmir":"Jammu and Kashmir",
                                    "A& N Islands":"Andaman and Nicobar Islands",
                                    "D & N Haveli":"Dadra and Nagar Haveli"}

district_name_mapping = {"Jagitial":"Jagtial", "Jangoan":"Jangaon", "Kumuram Bheem Asifabad":"Komaram Bheem Asifabad",
                         "Mahabubnagar": "Mahbubnagar", "Ranga Reddy": "Rangareddy", "Yadadri Bhuvanagiri":"Yadadri Bhongir",
                         "Jayashankar": "Jayashankar Bhupalapally",
                         "West Khasi Hills":"West khasi Hills", "East Jaintia Hills":"East Jainta Hills",
                        "Kalaburagi":"Gulbarga", "Belagavi":"Belgaum", "Bagalkote":"Bagalkot", "Ballari":"Bellary",
                        "Shivamogga":"Shimoga", "Chikkamagaluru":"Chikmagalur", "Bengaluru Rural":"Bangalore Rural",
                        "Bengaluru":"Bangalore", "Chamarajanagara":"Chamarajanagar", "Tumakuru":"Tumkur", "Mysuru":"Mysore","Vijayapura":"Bijapur",
                        "Gurugram":"Gurgaon", "Mahendragarh":"Narnaul","Charki Dadri":"Ch-Dadri",
                        "West Champaran": "Pashchim Champaran", "East Champaran":"Purba Champaran", "Kaimur Bhabhua":"Kaimur (Bhabua)", "Kaimur Bhabua":"Kaimur (Bhabua)",
                        "S.P.S. Nellore":"Nellore", "Chittoor":"chittoor",
                        "South 24 Parganas":"South Twenty Four Parganas", "Medinipur West":"Paschim Medinipur", "Medinipur East":"Purba Medinipur",
                        "Darjeeling":"Darjiling", "Purulia":"Puruliya", "North 24 Parganas":"North Twenty Four Parganas", "Hooghly":"Hugli",
                        "Howrah":"Haora", "Cooch Behar":"Koch Bihar",
                        "Gariaband":"Gariyaband", "Kabeerdham":"Kawardha", "Janjgir Champa":"Janjgir - Champa", "Bametara":"Bemetara",
                        "Baloda Bazar":"Balodabazar",
                        "North and Middle Andaman":"North & Middle Andman", "South Andaman":"South Andman","Nicobars":"Nicobar",
                        "Saraikela-kharsawan":"Saraikela-Kharsawan",
                        "Leh":"Leh(Ladakh)",
                        "Amroha":"J P Nagar (Amroha)", "Kheri":"Lakhimpur Kheri", "Kasganj":"Kanshiram Nagar", "Prayagraj":"Allahabad",
                        "Bhadohi":"Sant Ravidas Nagar (Bhadohi)", "Amethi":"C S M Nagar (Amethi)",
                        "Thoothukudi":"Thoothukkudi","Dadra and Nagar Haveli":"Uni District UT", 
                        "Khargone":"Khargone (West Nimar)", "Khandwa":"Khandwa (East Nimar)", "Ashoknagar":"Ashok Nagar",
                        "Kra Daadi":"Kra daadi", "Lower Dibang Valley":"Lower Dibang valley", "Upper Dibang Valley":"Dibang Valley",
                        "Haridwar":"Hardwar", "Noney":"None", "Pherzawl":"Pherjawl", "Sipahijala":"Sepahijala District","Gomati":"Gomati District",
                        "Khowai":"Khowai District", "Unokoti":"Unakoti District", "The Dangs":"Dang", "Chota Udaipur":"Chhota Udepur",
                        "Aravalli":"Arvalli",
                        "S.A.S. Nagar":"Mohali", "Sri Muktsar Sahib":"Muktsar","Jalore":"Jalor",
                        "Jhunjhunu":"Jhunjhunun", "Lahul and Spiti":"Lahul & Spiti","North  District":"North District"
                        }


district_name_resolution = ["Aurangabad", "Bijapur", "Raigarh", "Bilaspur", "Balrampur", "Hamirpur", "Pratapgarh"]

"""
district_name_unmapped_old = ["Muzaffarabad", "Mirpur", "Shi Yomi", "Pakke Kessang", "Pakke-Kessang", "Lepa Rada", "Pauri Garhwal",
                         "Charaideo", "Majuli", "Biswanath", "Hojai", "South Salmara Mancachar",
                         "Shahid Bhagat Singh Nagar", "West Karbi Anglong"]
"""

district_name_wiki_mapping = {"Sri Potti Sriramulu Nellore":"Nellore", "Kadapa":"Y.S.R.",
    "Dantewada":"Dakshin Bastar Dantewada", "Janjgir-Champa":"Janjgir - Champa","Kabirdham":"Kawardha", "Kanker":"Uttar Bastar Kanker",
    "Ahmedabad":"Ahmadabad", "Banaskantha": "Banas Kantha", "Dahod":"Dohad", "Devbhoomi Dwarka":"Devbhumi Dwarka",
    "Kutch":"Kachchh", "Mehsana":"Mahesana", "Panchmahal":"Panch Mahals", "Sabarkantha":"Sabar Kantha",
    "Charkhi Dadri":"Ch-Dadri", "Hissar":"Hisar", "Yamuna Nagar":"Yamunanagar",
    "Lahaul and Spiti":"Lahul & Spiti",
    "East Singhbhum":"Purbi Singhbhum", "West Singhbhum":"Pashchimi Singhbhum", "Hazaribag":"Hazaribagh", "Koderma":"Kodarma", "Seraikela Kharsawan":"Saraikela-Kharsawan",
    "Bangalore Urban":"Bangalore","Chamarajnagar":"Chamarajanagar","Chikkaballapur":"Chikkaballapura",
    "Narsinghpur":"Narsimhapur", "Ahmednagar":"Ahmadnagar", "Beed":"Bid", "Buldhana":"Buldana", "Gondia":"Gondiya", "Mumbai City":"Mumbai","Mumbai suburban":"Mumbai Suburban",
    "Raigad":"Raigarh (Maharashtra)","Ri Bhoi":"Ribhoi", "Angul":"Anugul", "Boudh (Bauda)":"Baudh", "Bargarh (Baragarh)":"Bargarh", "Balasore":"Baleshwar",
    "Debagarh (Deogarh)":"Debagarh", "Jajpur":"Jajapur", "Jagatsinghpur":"Jagatsinghapur","Kendujhar (Keonjhar)":"Kendujhar", "Nabarangpur":"Nabarangapur", "Subarnapur (Sonepur)":"Subarnapur",
    "Chittorgarh":"Chittaurgarh", "Dholpur":"Dhaulpur", "East Sikkim":"East District", "North Sikkim":"North District", "South Sikkim":"South District", "West Sikkim":"West District",
    "Kanchipuram":"Kancheepuram", "Kanyakumari":"Kanniyakumari", "Nilgiris":"The Nilgiris", "Tirupur":"Tiruppur",
    "Tiruvallur":"Thiruvallur","Tiruvarur":"Thiruvarur", "Komaram Bheem":"Komaram Bheem Asifabad","Jayashankar Bhupalpally":"Jayashankar Bhupalapally", "Medchal-Malkajgiri":"Medchal Malkajgiri",
    "Sepahijala":"Sepahijala District", "Bagpat":"Baghpat", "Barabanki":"Bara Banki", "Gautam Buddh Nagar":"Gautam Buddha Nagar",
    "Maharajganj":"Mahrajganj", "Raebareli":"Rae Bareli", "Shravasti":"Shrawasti","Bandipora":"Bandipore", "Baramulla":"Baramula",
    "Poonch":"Punch", "Shopian":"Shupiyan","Mahé":"Mahe","Pondicherry":"Puducherry",
    "Kaimur":"Kaimur (Bhabua)", "Sahibzada Ajit Singh Nagar":"Mohali","Jalore":"Jalor"
}

district_name_unmapped = ["Muzaffarabad", "Mirpur", "Shi Yomi", "Pakke Kessang", "Pakke-Kessang", "Lepa Rada", "Pauri Garhwal",
                         "Charaideo", "Majuli", "Biswanath", "Hojai", "South Salmara Mancachar", "South Salmara", "Bishwanath",
                         "Shahid Bhagat Singh Nagar", "West Karbi Anglong",
                         "Gaurela-Pendra-Marwahi", "Chachaura-Binaganj", "Maihar", "Niwari", "Nagda", "Noklak",
                         "Sahibzada Ajit Singh Nagar", "Jalore", "Chengalpattu", "Kallakurichi", "Mayiladuthurai",
                         "Ranipet","Tenkasi","Tirupattur", "Daman", "Diu", "Niwari"]

### Misc

In [ ]:
elif "No travel history" in notes:
            df.at[i, "cluster"] = "No travel history"
            df.at[i, "cluster_district"] = district
            df.at[i, "cluster_state"] = state
        
        elif "Co-passenger" in notes:
            df.at[i, "cluster"] = "Co-passenger"
            df.at[i, "cluster_district"] = district
            df.at[i, "cluster_state"] = state

        elif "Cab driver" in notes:
            df.at[i, "cluster"] = "Cab driver"
            df.at[i, "cluster_district"] = district
            df.at[i, "cluster_state"] = state
        
        elif "Doctor" in notes:
            df.at[i, "cluster"] = "Doctor"
            df.at[i, "cluster_district"] = district
            df.at[i, "cluster_state"] = state
        
        elif "Indian Army jawan" in notes:
            df.at[i, "cluster"] = "Indian Army jawan"
            df.at[i, "cluster_district"] = district
            df.at[i, "cluster_state"] = state